In [76]:
# input_path = "documentacion/1. Estatutos (2022).md"
# input_path = "documentacion/2. Programa Educativo de Grupo 2022 - 2025.md"
# input_path = "documentacion/3. Reglamento de Regimen Interno (2023).md"
input_path = "documentacion/4. Normas de Organización y Funcionamiento (2023).md"
output_path = "vector_db"

In [77]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [78]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [79]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## MARKDOWN LOADER

In [80]:
def read_md_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            md_content = file.read()
        return md_content
    except FileNotFoundError:
        print("El archivo no fue encontrado.")
        return None

# Ejemplo de uso
md_string = read_md_file(input_path)
md_string

'# 4. Normas de Organización y Funcionamiento (2023)\n\n## CAPÍTULO I. DISPOSICIONES GENERALES ÍNDICE\n\n### SECCIÓN PRIMERA. ÁMBITO DE APLICACIÓN DEL REGLAMENTO.\n\n#### Artículo 1.- Presentación\n\n```\nLa Asociación Grupo Scout Annapurna (GSA), en adelante “el Grupo”,\nconstituida al amparo de la legislación vigente se regirá por los Estatutos de la\nAsociación, por el Reglamento de Régimen Interno y por el presente\ndocumento de Normas de Organización y Funcionamiento.\n```\n#### Artículo 2.- Funcionamiento\n\n```\nEl presente documento de Normas de Organización y Funcionamiento\ndesarrolla los contenidos expresados en los Estatutos de la Asociación y del\nReglamento de Régimen Interno y en ningún caso podrá ir contra la filosofía y\nartículos de los citados.\n```\n#### Artículo 3.- Uso\n\n```\nEl presente Reglamento tiene carácter obligatorio para personas socias,\npersonas beneficiarias, familias, personas monitoras, personas educandas,\nantiguas personas pertenecientes al Grupo,

In [81]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# headers_to_split_on = [
#     ("#", "source"),
#     ("##", "capitulo"),
#     ("###", "apartado"),
#     ("####", "subapartado"),
# ]

# Fichero 1
# headers_to_split_on = [
#     ("#", "source"),
#     ("##", "capitulo"),
#     ("###", "articulo"),
#     ("####", "articulo"),
# ]

# Fichero 2
# headers_to_split_on = [
#     ("#", "source"),
#     ("##", "capitulo"),
#     ("###", "apartado"),
#     ("####", "subapartado"),
# ]

# Ficheros 3 y 4
headers_to_split_on = [
    ("#", "source"),
    ("##", "capitulo"),
    ("###", "seccion"),
    ("####", "articulo"),
]


markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
splits = markdown_splitter.split_text(md_string)
for doc in splits:
    print(doc.metadata)

{'source': '4. Normas de Organización y Funcionamiento (2023)', 'capitulo': 'CAPÍTULO I. DISPOSICIONES GENERALES ÍNDICE', 'seccion': 'SECCIÓN PRIMERA. ÁMBITO DE APLICACIÓN DEL REGLAMENTO.', 'articulo': 'Artículo 1.- Presentación'}
{'source': '4. Normas de Organización y Funcionamiento (2023)', 'capitulo': 'CAPÍTULO I. DISPOSICIONES GENERALES ÍNDICE', 'seccion': 'SECCIÓN PRIMERA. ÁMBITO DE APLICACIÓN DEL REGLAMENTO.', 'articulo': 'Artículo 2.- Funcionamiento'}
{'source': '4. Normas de Organización y Funcionamiento (2023)', 'capitulo': 'CAPÍTULO I. DISPOSICIONES GENERALES ÍNDICE', 'seccion': 'SECCIÓN PRIMERA. ÁMBITO DE APLICACIÓN DEL REGLAMENTO.', 'articulo': 'Artículo 3.- Uso'}
{'source': '4. Normas de Organización y Funcionamiento (2023)', 'capitulo': 'CAPÍTULO I. DISPOSICIONES GENERALES ÍNDICE', 'seccion': 'SECCIÓN SEGUNDA. NORMAS GENERALES.', 'articulo': 'Artículo 4.- Introducción'}
{'source': '4. Normas de Organización y Funcionamiento (2023)', 'capitulo': 'CAPÍTULO I. DISPOSICIONES

## VECTORS

In [82]:
from langchain.vectorstores import Chroma

In [83]:
# from langchain.embeddings import HuggingFaceEmbeddings

# embedding = HuggingFaceEmbeddings()

In [84]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [85]:
vectordb = Chroma(persist_directory=output_path, embedding_function=embedding)
print(vectordb._collection.count())

154


In [86]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=output_path
)

In [87]:
print(vectordb._collection.count())

174


## CLASE

In [88]:
# # input_path = "pipeline_files/2_transcribed_audio/gestion_de_requisitos_con_Redmine.json"
# input_path = "pipeline_files/2_transcribed_audio_coffee"
# # output_path = "pipeline_files/3_vectordb"
# output_path = "pipeline_files/3_vectordb_coffee"
# chunk_duration = 10 # in seconds
# reset_db = True

In [89]:
# import json
# from langchain.docstore.document import Document
# from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
# import os
# import shutil
# import torch


# class VectorDBGenerator:
#     def __init__(self, output_path, reset_db=True):
#         self.output_path = output_path
#         if reset_db:
#             self.__delete_db()
#         self.device = "cuda" if torch.cuda.is_available() else "cpu"
#         self.embeddings = HuggingFaceEmbeddings()

#     def generate_vectordb(self, input_path, chunk_duration):
#         json_data = self.__read_json_file(input_path)
#         chunks = self.__chunk_aggregator(json_data, chunk_duration)
#         documents = self.__generate_documents(chunks)
#         vectordb = self.__generate_vectors(documents)
#         return vectordb
    

#     def __read_json_file(self, file_path):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             data = json.load(file)
#         return data


#     def __chunk_aggregator(self, data, chunk_duration):
#         # duration_chunk, chunk_text, chunk_start, chunk_end = 0, "", "00:00:00,000", "00:00:00,000"
#         duration_chunk, chunk_text, chunk_start, chunk_end = 0, "", "00:00:00.000", "00:00:00.000"
#         chunks = []
#         for el in data:
#             start = float(self.__time_to_seconds(el["start"]))
#             end = float(self.__time_to_seconds(el["end"]))
#             duration_chunk += end - start
#             # chunk_text += el["text"] + " "
#             chunk_text += el["content"] + " "
#             if duration_chunk >= chunk_duration:
#                 chunk_end = el["end"]
#                 # chunks.append({"filename": el["filename"], "start": chunk_start, "end": chunk_end, "text": chunk_text})
#                 chunks.append({"episode": el["episode"], "start": chunk_start, "end": chunk_end, "content": chunk_text})
#                 chunk_start = el["end"]
#                 chunk_text = ""
#                 duration_chunk = 0
#         return chunks
    
#     def __time_to_seconds(self, time_str):
#         time_components = time_str.split(':')
#         hours = int(time_components[0])
#         minutes = int(time_components[1])
#         # seconds, milliseconds = map(float, time_components[2].split(','))
#         seconds, milliseconds = map(float, time_components[2].split('.'))
#         total_seconds = hours * 3600 + minutes * 60 + seconds + milliseconds / 1000
#         return total_seconds
    

#     def __generate_documents(self, data):
#         splits = []
#         for el in data:
#             metadata = {}
#             # metadata["filename"] = el["filename"]
#             # metadata["start"] = el["start"]
#             # metadata["end"] = el["end"]
#             # doc =  Document(page_content=el["text"], metadata=metadata)
#             metadata["filename"] = el["episode"]
#             metadata["start"] = el["start"]
#             metadata["end"] = el["end"]
#             doc =  Document(page_content=el["content"], metadata=metadata)
#             splits.append(doc)
#         return splits
    
#     def __generate_vectors(self, documents):
#         vectordb = Chroma.from_documents(
#             documents=documents,
#             embedding=self.embeddings,
#             persist_directory=self.output_path
#         )
#         return vectordb
    
#     def __delete_db(self):
#         if os.path.exists(self.output_path):
#             shutil.rmtree(self.output_path)
#         os.makedirs(self.output_path)


In [90]:
# generator = VectorDBGenerator(output_path, reset_db)
# if os.path.isdir(input_path):
#     for file_name in os.listdir(input_path):
#         if file_name.endswith(".json"):
#             file_path = os.path.join(input_path, file_name)
#             generator.generate_vectordb(file_path, chunk_duration)
# else:
#     generator.generate_vectordb(input_path, chunk_duration)

In [91]:
# from tqdm import tqdm

# generator = VectorDBGenerator(output_path, reset_db)
# if os.path.isdir(input_path):
#     for file_name in tqdm(os.listdir(input_path), desc="Processing files"):
#         if file_name.endswith(".json"):
#             file_path = os.path.join(input_path, file_name)
#             generator.generate_vectordb(file_path, chunk_duration)
# else:
#     generator.generate_vectordb(input_path, chunk_duration)
